In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os

from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

import hasasia.sensitivity as hsen
import hasasia.sim as hsim
import hasasia.skymap as hsky

In [ ]:
SHAPE = None
NREALS = 500
NFREQS = 40
NLOUDEST = 10

SAVEFIG = False
TOL=0.01
MAXBADS=5

RED_GAMMA = None
RED2WHITE = None

NVARS = 21

NPSRS = 40
NSKIES = 100
TARGET = 'gsmf_phi0' # EDIT AS NEEDED
TITLE = plot.PARAM_KEYS[TARGET]  # EDIT AS NEEDED

In [ ]:
def get_data(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz',     
):
    load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data']
    params = file['params']
    file.close()
    print(target, "got data")

    return data, params

In [ ]:
data, params = get_data(TARGET)

In [ ]:
var=10

fobs_cents = data[var]['fobs_cents']
dur = 1.0 / fobs_cents[0]
cad = 1.0 / (2.0 * fobs_cents[-1])
hc_ss = data[var]['hc_ss']
hc_bg = data[var]['hc_bg']

In [ ]:
phis = None
thetas = None
npsrs = NPSRS
red2white = RED2WHITE
sigstart = 1.5e-7

# randomize pulsar positions
if phis is None: phis = np.random.uniform(0, 2*np.pi, size = npsrs)
if thetas is None: thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
sigma = sigstart
# if red2white is not None:
#     red_amp = _white_noise(cad, sigma) * red2white

psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigma,
                phi=phis, theta=thetas)

In [ ]:
dp_bg = detstats.detect_bg_pta(psrs, fobs_cents, hc_bg=hc_bg[:,np.newaxis], hc_ss=hc_ss[:,np.newaxis,:],
                            red_amp=None, red_gamma=None, ss_noise=False)[0]
print(holo.utils.stats(dp_bg))

In [ ]:
spectra = []
for psr in psrs:
    sp = hsen.Spectrum(psr, freqs=fobs_cents)
    sp.NcalInv
    spectra.append(sp)

In [ ]:
sc_ss = hsen.DeterSensitivityCurve(spectra)
sc_bg = hsen.GWBSensitivityCurve(spectra)

In [ ]:
noise_ss = sc_ss.h_c
noise_bg = sc_bg.h_c

In [ ]:
fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN)
xx = fobs_cents*YR
ax.plot(xx, noise_bg, label='BG Sensitivity', linestyle='--', color='tab:blue')
ax.plot(xx, noise_ss, label='SS Sensitivity', linestyle='--', color='tab:pink')

ax.fill_between(xx, *np.percentile(hc_bg, (25,75), axis=-1), 
                color='tab:blue', alpha=0.2)
ax.plot(xx, np.median(hc_bg, axis=-1), label='hc_BG', color='tab:blue')

ax.fill_between(xx, *np.percentile(hc_ss[:,:,0], (25, 75), axis=-1),
                color='tab:pink', alpha=0.2)
ax.plot(xx, np.median(hc_ss[:,:,0], axis=-1), label='hc_SS', color='tab:pink')
ax.legend()